# IMPORT

# DATA TRANSFORMATION
Hereafter we transform the csv in parquet for perfomance improvement. se 

In [1]:
catalog.list()


[
    'client',
    'electricity_prices',
    'forecast_weather',
    'gas_prices',
    'historical_weather',
    'train',
    'county_id_to_name_map',
    'client_eg',
    'electricity_prices_eg',
    'forecast_weather_eg',
    'gas_prices_eg',
    'historical_weather_eg',
    'revealed_targets',
    'sample_submission',
    'test',
    'metrics',
    'parameters'
]

In [4]:
catalog.load("electricity_prices")

[11/09/23 16:30:27] INFO     Loading data from 'electricity_prices' (CSVDataset)...             ]8;id=145041;file:///home/ensai/.pyenv/versions/3.9.18/envs/enefit/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=538921;file:///home/ensai/.pyenv/versions/3.9.18/envs/enefit/lib/python3.9/site-packages/kedro/io/data_catalog.py#502\502]8;;\

,forecast_date,euros_per_mwh,origin_date,data_block_id
0,2021-09-01 00:00:00,92.51,2021-08-31 00:00:00,1
1,2021-09-01 01:00:00,88.90,2021-08-31 01:00:00,1
2,2021-09-01 02:00:00,87.35,2021-08-31 02:00:00,1
3,2021-09-01 03:00:00,86.88,2021-08-31 03:00:00,1
4,2021-09-01 04:00:00,88.43,2021-08-31 04:00:00,1
...,...,...,...,...
15281,2023-05-30 19:00:00,82.10,2023-05-29 19:00:00,637
15282,2023-05-30 20:00:00,150.85,2023-05-29 20:00:00,637
15283,2023-05-30 21:00:00,82.10,2023-05-29 21:00:00,637
15284,2023-05-30 22:00:00,82.09,2023-05-29 22:00:00,637


In [5]:
context.project_path

PosixPath('/home/ensai/dev/enefit')